In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers,Model
from keras.layers import Activation, Dense,Conv2D,Dropout,Activation,Flatten,MaxPooling2D
from tensorflow.keras.models import Sequential
import imblearn
from imblearn.over_sampling import RandomOverSampler,SMOTE
import pandas as pd
import numpy as np
import os
from os import listdir
from os.path import isfile, join
import librosa
from sklearn.model_selection import train_test_split

import numpy as np
import scipy.io.wavfile as wav
from scipy.io.wavfile import write
from io import BytesIO
from base64 import b64decode
from google.colab import output
import IPython.display as ipd

from IPython.display import Javascript





In [17]:
train_path='/content/sample_data/train/'
test_path='/content/sample_data/test/'
model_path='D:\\upwork\\New folder (2)\\audio_classification'
# classes=['blue','change_color','green','high','low','noise','off','on','party','purple','red','sleep','unknown','wakeup','white','yellow']
classes=['firefly', 'off', 'party', 'red', 'sleep', 'white', 'yellow']

In [11]:
len(classes)

7

In [18]:
def features_extractorcnn(file_name):
    audio, sample_rate = librosa.load(file_name) 
    mfccs_features =librosa.feature.melspectrogram(y=audio,sr=sample_rate)
    #mfccs_scaled_features = np.mean(mfccs_features.T,axis=0)
    
    return mfccs_features.reshape(128,44,1)

In [21]:
x=np.array([features_extractorcnn(train_path+'/'+f) for f in listdir(train_path) if isfile(join(train_path, f))])
x_test=np.array([features_extractorcnn(test_path+'/'+f) for f in listdir(test_path) if isfile(join(test_path, f))])

In [22]:
y=np.array([f.split('.')[0] for f in listdir(train_path) if isfile(join(train_path, f))])
y=np.array(pd.get_dummies(y))


In [23]:
y.shape

(448, 7)

In [24]:
x_train,x_val,y_train,y_val=train_test_split(x,y,test_size=0.2,random_state=0)

In [25]:
oversample = RandomOverSampler(sampling_strategy='minority')
#x_over, y_over = oversample.fit_resample(x_train, y_train)
x_over, y_over = oversample.fit_resample(x_train.reshape(len(x_train),-1), y_train)
x_over=x_over.reshape(len(x_over),128,44,1)


In [26]:
y_over.shape

(386, 7)

In [89]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=3)


mobile=tf.keras.applications.mobilenet.MobileNet( input_shape=(128,44,1),include_top=False,weights=None)
x=mobile.layers[-1].output
output=Dense(units=512,activation="softmax")(x)
mobilnet_model=Model(inputs=mobile.input,outputs=output)
for layer in mobilnet_model.layers[:-1]:
    layer.trainable=False

mobilnet_model = keras.models.Sequential([
    mobilnet_model,
    layers.Flatten(),
    # tf.keras.layers.GlobalAveragePooling2D(),
    # keras.layers.Dropout(0.35),
    # keras.layers.Dense(128, activation='relu'),
     keras.layers.Dropout(0.35),
     keras.layers.Dense(7, activation='softmax')                                        
])
mobilnet_model.compile(optimizer='adam',
            loss='categorical_crossentropy',
            metrics=['accuracy'])
mobilnet_model.fit(x_train,y_train, epochs=7, validation_data=(x_val, y_val))#,callbacks=[callback]


Epoch 1/7
12/12 [==============================] - 2s 56ms/step - loss: 1.9442 - accuracy: 0.1453 - val_loss: 1.9408 - val_accuracy: 0.1778
Epoch 2/7
12/12 [==============================] - 0s 16ms/step - loss: 1.9399 - accuracy: 0.1899 - val_loss: 1.9377 - val_accuracy: 0.1889
Epoch 3/7
12/12 [==============================] - 0s 18ms/step - loss: 1.9367 - accuracy: 0.1844 - val_loss: 1.9351 - val_accuracy: 0.1889
Epoch 4/7
12/12 [==============================] - 0s 16ms/step - loss: 1.9332 - accuracy: 0.1844 - val_loss: 1.9326 - val_accuracy: 0.1889
Epoch 5/7
12/12 [==============================] - 0s 13ms/step - loss: 1.9308 - accuracy: 0.1844 - val_loss: 1.9306 - val_accuracy: 0.1889
Epoch 6/7
12/12 [==============================] - 0s 13ms/step - loss: 1.9297 - accuracy: 0.1844 - val_loss: 1.9295 - val_accuracy: 0.1889
Epoch 7/7
12/12 [==============================] - 0s 14ms/step - loss: 1.9286 - accuracy: 0.1844 - val_loss: 1.9282 - val_accuracy: 0.1889


In [90]:
mobilnet_model.predict(x_test)

array([[0.12139317, 0.1664863 , 0.14855033, 0.14388622, 0.124913  ,
        0.12955   , 0.16522093],
       [0.12139317, 0.1664863 , 0.14855033, 0.14388622, 0.124913  ,
        0.12955   , 0.16522093],
       [0.12139317, 0.1664863 , 0.14855033, 0.14388622, 0.124913  ,
        0.12955   , 0.16522093],
       [0.12139317, 0.1664863 , 0.14855033, 0.14388622, 0.124913  ,
        0.12955   , 0.16522093],
       [0.12139317, 0.1664863 , 0.14855033, 0.14388622, 0.124913  ,
        0.12955   , 0.16522093],
       [0.12139317, 0.1664863 , 0.14855033, 0.14388622, 0.124913  ,
        0.12955   , 0.16522093],
       [0.12139317, 0.1664863 , 0.14855033, 0.14388622, 0.124913  ,
        0.12955   , 0.16522093],
       [0.12139317, 0.1664863 , 0.14855033, 0.14388622, 0.124913  ,
        0.12955   , 0.16522093],
       [0.12139317, 0.1664863 , 0.14855033, 0.14388622, 0.124913  ,
        0.12955   , 0.16522093],
       [0.12139317, 0.1664863 , 0.14855033, 0.14388622, 0.124913  ,
        0.12955   , 0.1

In [91]:
mobilnet_model.summary()

Model: "sequential_34"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 model_37 (Functional)       (None, 4, 1, 512)         3753088   
                                                                 
 flatten_34 (Flatten)        (None, 2048)              0         
                                                                 
 dropout_46 (Dropout)        (None, 2048)              0         
                                                                 
 dense_83 (Dense)            (None, 7)                 14343     
                                                                 
Total params: 3,767,431
Trainable params: 539,143
Non-trainable params: 3,228,288
_________________________________________________________________


In [92]:
classes=['firefly', 'off', 'party', 'red', 'sleep', 'white', 'yellow']
predictions=[classes[i] for i in np.argmax(mobilnet_model.predict(x_test.reshape(len(x_test-1),128,44,1)),axis=1)]
test_classes=[f.split('.')[0] for f in listdir(test_path) if isfile(join(test_path, f))]
mydf=pd.DataFrame(data={'real':test_classes, 'pred':predictions})
mydf

,real,pred
0,party,off
1,party,off
2,party,off
3,yellow,off
4,white,off
...,...,...
98,sleep,off
99,off,off
100,red,off
101,firefly,off


In [33]:
def predictSound(soundFile):
    x=features_extractorcnn(soundFile)
    i=np.argmax(mobilnet_model.predict(x[np.newaxis,...]))
    return classes[i]
    
    

In [62]:
predictSound('/content/sample_data/test/firefly.2v6b5tfg.ingestion-7558d88f45-k9jh8.wav')

'off'

In [61]:
from io import BytesIO
from base64 import b64decode
from google.colab import output
import IPython.display as ipd

from IPython.display import Javascript




def record(sec=1):
  RECORD = """
const sleep  = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.srcElement.result)
  reader.readAsDataURL(blob)
})
var record = time => new Promise(async resolve => {
  stream = await navigator.mediaDevices.getUserMedia({ audio: true })
  recorder = new MediaRecorder(stream)
  chunks = []
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.start()
  await sleep(time)
  recorder.onstop = async ()=>{
    blob = new Blob(chunks)
    text = await b2text(blob)
    resolve(text)
  }
  recorder.stop()
})
"""
  print("Speak Now...")
  display(Javascript(RECORD))
  # sec += 0.03
  sec += 1.0
  s = output.eval_js('record(%d)' % (sec*1000))
  print("Done Recording !")
  b = b64decode(s.split(',')[1])
  ipd.display(ipd.Audio(b))
  with open('audio.wav','wb') as f:
    f.write(b)
  audio, sample_rate = librosa.load('audio.wav')
  os.remove("audio.wav") 
  mfccs_features =librosa.feature.melspectrogram(y=audio[200:22250],sr=sample_rate)
  return mfccs_features.reshape(128,44,1)

In [63]:

def predictRecord():
    
    x=record() 
    i=np.argmax(mobilnet_model.predict(x[np.newaxis,...]))
    return classes[i]

In [64]:
predictRecord()

Speak Now...


<IPython.core.display.Javascript object>

Done Recording !


/usr/local/lib/python3.7/dist-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


'off'